#### base版本

In [ ]:
from paddleocr import PaddleOCRVL  # pyright: ignore[reportMissingImports]

pipeline = PaddleOCRVL()

output = pipeline.predict(r"e:\OneDrive\project_code\output\vision_temp\pdf_images_25840_642\page_098.png")
for res in output:
    res.print() ## 打印预测的结构化输出
    res.save_to_json(save_path="output") ## 保存当前图像的结构化json结果
    res.save_to_markdown(save_path="output") ## 保存当前图像的markdown格式的结果

c:\Users\cheng\miniconda3\envs\paddle_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\cheng\miniconda3\envs\paddle_env\lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-DocLayoutV2', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\cheng\.paddlex\official_models\PP-DocLayoutV2`.
Creating model: ('PaddleOCR-VL-0.9B', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\cheng\.paddlex\official_models\PaddleOC

RuntimeError: Exception from the 'cv' worker: Image read Error: E:\OneDrive\商汤\商汤FVTPL2506\4_资料清单\客户答复\09_卢卡杭州\卢卡 to 商汤\杭州卢卡报表0331\杭州卢卡报表0331_01.jpg

#### 用8000端口api形式

In [3]:
from LLMkit.modules.vision.ocr_client import OCRClient  # pyright: ignore[reportMissingImports]
from pathlib import Path
from LLMkit import output_address  # pyright: ignore[reportMissingImports]
client = OCRClient("http://100.107.137.48:8000")

In [ ]:
# 使用你的文件路径
result = client.recognize_image(
    image_path=r"e:\OneDrive\project_code\output\vision_temp\pdf_images_25840_642\page_098.png",
    save_markdown=True,
    save_json=False,
    output_dir="output"
)

print(result)

{'status': 'error', 'message': 'Failed to recognize image: 400 Client Error: Bad Request for url: http://100.107.137.48:8000/ocr/recognize'}


In [3]:
md_path = result.get("markdown_path")
if md_path:
    md_path_full = Path(output_address+md_path[7:])

import os
os.startfile(md_path_full)

#### 批量整合md

pdf转图片

In [4]:
from LLMkit.modules.vision.document_processor import DocumentProcessor

app = DocumentProcessor(dpi = 200)
png_result = app.process_document(r"E:\OneDrive\project_code\杭州卢卡报表0331.pdf")

INFO:LLMkit.modules.vision.document_processor:开始处理文档: E:\OneDrive\project_code\杭州卢卡报表0331.pdf
INFO:LLMkit.modules.vision.document_processor:已处理第 1 页: e:\OneDrive\project_code\output\vision_temp\pdf_images_44284_6959\page_001.png
INFO:LLMkit.modules.vision.document_processor:已处理第 2 页: e:\OneDrive\project_code\output\vision_temp\pdf_images_44284_6959\page_002.png
INFO:LLMkit.modules.vision.document_processor:已处理第 3 页: e:\OneDrive\project_code\output\vision_temp\pdf_images_44284_6959\page_003.png


图片处理合并

In [5]:
import os

dir_path = r"e:\OneDrive\project_code\output\vision_temp\pdf_images_25840_642"

dir_path = png_result[0]["output_dir"]
file_list = [os.path.join(dir_path, f) for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))]
print(file_list)

['e:\\OneDrive\\project_code\\output\\vision_temp\\pdf_images_44284_6959\\page_001.png', 'e:\\OneDrive\\project_code\\output\\vision_temp\\pdf_images_44284_6959\\page_002.png', 'e:\\OneDrive\\project_code\\output\\vision_temp\\pdf_images_44284_6959\\page_003.png']


In [ ]:
content = []

for x in file_list[0:1]:
    # 使用你的文件路径
    result = client.recognize_image(
        image_path=x,
        save_markdown=True,
        save_json=False,
        output_dir="output"
    )

    md_path = result.get("markdown_path")
    if md_path:
        md_content = Path(output_address+md_path[7:]).read_text(encoding="utf-8")
    content.append(md_content)

print(content)

In [14]:
md_output_path = "merged_output.md"

with open(md_output_path, "w", encoding="utf-8") as f:
    for md in content:
        if md:
            f.write(md)
            f.write("\n\n")  # 可选：让每个md之间有空行，便于分隔

print(f"已保存合并后的markdown到: {md_output_path}")

import os

if md_path:
    os.startfile(md_output_path)

已保存合并后的markdown到: merged_output.md
